### 모듈, 함수정의 <br>

In [1]:
import os
import sys
import struct
import numpy as np

In [2]:
# 파일 불러오기
def read_file(filename):
  contents = bytes()

  with open(filename, "rb") as f:
    f.seek(0, 2)
    num_points = int(f.tell() / 4)
    f.seek(0, 0)
    contents = f.read()

  arr = [struct.unpack('<I', contents[4 * i:4 * i + 4])[0] for i in range(num_points)]

  return arr

In [3]:
# 파일 저장하기
def save_file(filename, labels):
  arr = [struct.pack('<I', label) for label in labels]

  with open(filename, "bw") as f:
    for a in arr:
      f.write(a)

### 경로 지정, 파일 불러오기 <br>

In [4]:
os.getcwd()

'/home/alien4/바탕화면'

In [5]:
# 바탕화면에서 주피터노트북 시작하면 바탕화면 경로
# os.getcwd() 이용해도됨
cur_path = os.getcwd()

# Lane 라벨링 할 디렉토리명
work_dir = "210205_A01"

# label,velodyne
labels_path = os.path.join(cur_path, work_dir, "labels")
velodyne_path = os.path.join(cur_path, work_dir, "velodyne")

In [6]:
# 파일경로 지정
for file in os.listdir(labels_path):
    if file.endswith(".label"):
        label_file = os.path.join(labels_path, file)
        
for file in os.listdir(velodyne_path):
    if file.endswith(".bin"):
        bin_file = os.path.join(velodyne_path, file)

In [7]:
#불러오기
labels=read_file(label_file)
bins=read_file(bin_file)

### 파일 수정, 저장 <br>

In [8]:
# 점의 개수
num_points = len(labels)
num_points

8544088

In [9]:
# 광도 정보들 brightness에 저장
brightness=[]
x=[]
y=[]
for i in range(num_points):
    x.append(bins[4*i])
    y.append(bins[4*i+1])
    brightness.append(bins[4*i+3])

In [13]:
# 빛의 세기 파라미터
intensity=1060000000 # default: intensity=1050000000

In [14]:
# 밝은 애들 모으기
high_intensity=[]
for i in range(num_points):
    if (brightness[i]>intensity) and (labels[i]==40):
        high_intensity.append(i)
len(high_intensity)

274466

In [40]:
array_x = np.array(x)
array_y = np.array(y)

In [ ]:
for i in range(num_points):
    if labels[i]!=40:
        array_x[i]=-1
        array_y[i]=-1

In [41]:
lane=[]
for i in high_intensity:
    same_x = set(np.where(array_x==x[i])[0])
    same_y = set(np.where(array_y==y[i])[0])
    same = same_x.intersection(same_y)
    same = list(same)
    lane.append(same)

KeyboardInterrupt: 

In [ ]:
lane_points=[]
for i in range(len(lane)):
    for j in lane[i]:
        lane_points.append(j)

In [ ]:
# 보라색중에 밝은 보라색을 차선으로 바꾸기
for i in lane_points:
    # 40=road , 60=lane
    labels[i]=60

In [ ]:
#저장
save_file(label_file,labels)